In [2]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [23]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [24]:

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_

# Test set
X_test_transformed = one_hot_enc.transform(X_test)
y_test_enc = le.transform(y_test)

In [25]:
os.getpid()

877

In [26]:
%%time
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')

rng = np.random.RandomState(31337)

kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for (i, (train_index, test_index)) in enumerate(kf.split(X)):    
    # preproc
    (X_transformed,
         one_hot_enc, le,
         y_enc) = pv1.preprocess(X[train_index], y[train_index], 
                             neighborhoods)
    
    xgb_model = xgb.XGBClassifier(objective='multi:softprob'
                                 ).fit(X_transformed, 
                                        y_enc, verbose=True)
    fu.log(workdir, f'[{i}] Done fit.')
    
    bundle_loc = f'{workdir}/bundle_{i}.joblib'
    joblib.dump({'model': xgb_model}, bundle_loc)
    #
    X_test_transformed = one_hot_enc.transform(X[test_index])
    actuals = le.transform(y[test_index]); len(actuals)
    
    predictions = xgb_model.predict(X_test_transformed)
    confusion = confusion_matrix(actuals, predictions)
    acc = accuracy_score(actuals, predictions)
    
    y_prob_vec = fu.predict_proba(X_test_transformed, bundle_loc=bundle_loc)
    # xgb_model.predict_proba(X_test_transformed)
    fu.log(workdir, f'[{i}] Done fu.predict_proba')
    
    
    logloss = fu.big_logloss(actuals, y_prob_vec, list(range(len(labels))))
    fu.log(workdir, f'[{i}] Done big_logloss, loss={logloss}.')
                          
    # save full now though
    joblib.dump({'model': xgb_model,
                'metrics': {'confusion': confusion,
                           'validation_logloss': logloss,
                           'validation_acc': acc}}, bundle_loc)
    fu.log(workdir, f'[{i}] dumped bundle to {bundle_loc}')
                             

/opt/program/artifacts/2020-06-20T215346Z
CPU times: user 20min 19s, sys: 8.64 s, total: 20min 27s
Wall time: 23min 30s


In [27]:
bundle_loc = '/opt/program/artifacts/2020-06-20T215346Z/bundle_0.joblib'
bundle = joblib.load(bundle_loc)
bundle

{'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=3,
               min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, objective='multi:softprob', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1),
 'metrics': {'confusion': array([[   0,   49,   12, ...,    0,    0,  143],
         [   0, 1112,   13, ...,    0,    0,   39],
         [   0,    1,  283, ...,    0,    0,  305],
         ...,
         [   0,    0,    7, ...,    0,    0,   73],
         [   0,  567,   27, ...,    0,    0,   57],
         [   0,   21,  163, ...,    0,    0, 2064]]),
  'validation_logloss': 3.3088296090867195,
  'validation_acc': 0.15462832101833496}}

In [28]:
bundle_loc = '/opt/program/artifacts/2020-06-20T215346Z/bundle_1.joblib'
bundle = joblib.load(bundle_loc)
bundle

{'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=3,
               min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, objective='multi:softprob', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1),
 'metrics': {'confusion': array([[ 193,   64,   14, ...,    0,    0,  150],
         [  27, 1167,   16, ...,    0,    0,   28],
         [   4,    6,  379, ...,    0,    0,  236],
         ...,
         [   0,    0,   17, ...,    0,    0,   46],
         [  67,  599,   24, ...,    0,    0,   78],
         [  19,    9,  196, ...,    0,    0, 2017]]),
  'validation_logloss': 3.306871724048802,
  'validation_acc': 0.1541161182619253}}

In [29]:
!top

=top - 22:39:42 up 4 days, 21:42,  0 users,  load average: 0.41, 0.31, 0.56
Tasks:   6 total,   1 running,   5 sleeping,   0 stopped,   0 zombie
%Cpu(s):  6.0 us,  0.7 sy,  0.0 ni, 93.1 id,  0.2 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem :  2046844 total,   634640 free,  1234124 used,   178080 buff/cache
KiB Swap:  1048572 total,    28140 free,  1020432 used.   650456 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
  667 root      20   0 1147296  21832   5456 S   5.9  1.1   7:40.77 jupyter-no+ 
  877 root      20   0 2782904 1.076g  11120 S   5.9 55.1  21:23.56 python      
    1 root      20   0   18236      4      0 S   0.0  0.0   0:00.11 bash        
  897 root      20   0  243148   3360   1240 S   0.0  0.2   0:00.12 python      
  927 root      20   0    4500    748    680 S   0.0  0.0   0:00.55 sh          
  928 root      20   0   36852   3080   2672 R   0.0  0.2   0:00.00 top         
top - 22:39:45 up 4 days, 21:42,  0 users,  load avera

In [30]:
os.getpid()

877

- Showing `1.076g` resident memory used by my notebook `pid=877` 
- This is anecdotally feeling better than before.
- Oh and no crashes here. May be due to single notebook running, or might be the 

In [31]:
from shlex import join

ImportError: cannot import name 'join' from 'shlex' (/opt/conda/lib/python3.7/shlex.py)

In [40]:
#import subprocess
!ps -p 877 -o pid,ppid,pmem,rss


  PID  PPID %MEM   RSS
  877   667 55.7 1142136


In [51]:
out = subprocess.check_output(["ps", "-p", "877", "-o", "pid,ppid,pmem,rss"])
print(out)
pid, ppid, pmem, rss = out.decode('utf-8').split('\n')[1].strip().split()
print(f'{pid}, {pmem}, {rss}')
print(int(rss)/1024/1024) 

b'  PID  PPID %MEM   RSS\n  877   667 55.8 1142288\n'
877, 55.8, 1142288
1.0893707275390625


In [52]:
def get_my_memory():
    mypid = os.getpid()
    out = subprocess.check_output(["ps", "-p", f"{mypid}", "-o", "pid,ppid,pmem,rss"])

    pid, ppid, pmem, rss = out.decode('utf-8').split('\n')[1].strip().split()
    # print(f'{pid}, {pmem}, {rss}')
    gigs = int(rss)/1024/1024
    assert int(pid) == mypid
    return {'pmem': pmem, 'rss': gigs}
get_my_memory()

{'pmem': '55.8', 'rss': 1.0894660949707031}